In [1]:
import pandas as pd
import tqdm
import numpy as np
from EduData import get_data
from sklearn.model_selection import KFold   # 不弄k-fold交叉验证就不用这个
import pickle
import os
import random

In [2]:
# 定义数据存放base路径
basePath='../../data/ednet/'

In [3]:
problem_detail = pd.read_csv(
    basePath+'EdNet/EdNet-Contents/contents/questions.csv',
    usecols=['question_id', 'bundle_id', 'correct_answer', 'tags']
)
problem_detail=problem_detail[problem_detail['tags']!='-1'] # tags=-1的记录被我视为异常，不取记录


In [4]:
# 建立练习映射,编号从1~n
raw_problem=problem_detail.question_id.unique().tolist()
num_problem=len(raw_problem)
problems={p:i+1 for i,p in enumerate(raw_problem)}
print("number of problems: %d" % num_problem)
np.save(basePath+'map/eMap.npy',problems)
print(problems)

number of problems: 12372
{'q1': 1, 'q2': 2, 'q3': 3, 'q4': 4, 'q5': 5, 'q6': 6, 'q7': 7, 'q8': 8, 'q9': 9, 'q10': 10, 'q11': 11, 'q12': 12, 'q13': 13, 'q14': 14, 'q15': 15, 'q16': 16, 'q17': 17, 'q18': 18, 'q19': 19, 'q20': 20, 'q21': 21, 'q22': 22, 'q23': 23, 'q24': 24, 'q25': 25, 'q26': 26, 'q27': 27, 'q28': 28, 'q29': 29, 'q30': 30, 'q31': 31, 'q32': 32, 'q33': 33, 'q34': 34, 'q35': 35, 'q36': 36, 'q37': 37, 'q38': 38, 'q39': 39, 'q40': 40, 'q41': 41, 'q42': 42, 'q43': 43, 'q44': 44, 'q45': 45, 'q46': 46, 'q47': 47, 'q48': 48, 'q49': 49, 'q50': 50, 'q51': 51, 'q52': 52, 'q53': 53, 'q54': 54, 'q55': 55, 'q56': 56, 'q57': 57, 'q58': 58, 'q59': 59, 'q60': 60, 'q61': 61, 'q62': 62, 'q63': 63, 'q64': 64, 'q65': 65, 'q66': 66, 'q67': 67, 'q68': 68, 'q69': 69, 'q70': 70, 'q71': 71, 'q72': 72, 'q73': 73, 'q74': 74, 'q75': 75, 'q76': 76, 'q77': 77, 'q78': 78, 'q79': 79, 'q80': 80, 'q81': 81, 'q82': 82, 'q83': 83, 'q84': 84, 'q85': 85, 'q86': 86, 'q87': 87, 'q88': 88, 'q89': 89, 'q90': 90, '

In [5]:
#将problem_id替换成eMap的value
problem_detail['question_id']=problem_detail['question_id'].map(problems)

In [6]:
# 建立技能映射，编号从0~n-1
tags_list=problem_detail.tags.unique().tolist()
print(tags_list)
tags_unique=[]
for tags in tags_list:
    tag_list=tags.split(';')
    for tag in tag_list:
        tag=int(tag)
        if tag not in tags_unique:
            tags_unique.append(tag)
tags_unique.sort()
print(tags_unique)

['1;2;179;181', '15;2;182', '14;2;179;183', '9;2;179;184', '8;2;179;181', '9;2;179;182', '11;7;179;183', '20;21;179;184', '13;2;179;183', '17;7;182', '15;2;181', '1;2;182', '1;2;183', '8;2;184', '14;2;181', '20;21;182', '19;2;183', '5;2;179;184', '20;21;183', '10;7;179;182', '9;2;179;181', '6;7;179;182', '8;2;183', '6;7;184', '6;7;181', '6;7;182', '4;2;179;183', '4;2;183', '13;2;179;182', '3;2;179;182', '5;2;179;181', '6;7;183', '8;2;182', '3;2;182', '10;7;179;183', '11;7;181', '4;2;181', '5;2;179;182', '16;2;183', '13;2;179;184', '17;7;179;182', '6;7;179;183', '22;2;182', '15;2;179;182', '20;21;179;183', '17;7;183', '14;2;179;182', '17;7;185', '4;2;185', '9;2;181', '12;2;183', '15;2;185', '22;2;185', '1;2;185', '5;2;179;183', '11;7;183', '17;7;184', '8;2;185', '6;7;185', '20;21;181', '15;2;183', '13;2;185', '10;7;181', '12;2;185', '14;2;185', '4;2;182', '22;2;183', '1;2;179;184', '18;7;179;182', '22;2;181', '22;2;179;183', '4;2;184', '10;7;182', '12;2;182', '13;2;181', '14;2;183', '10

In [7]:
# 建立技能映射，编号从0~n-1
num_skill = len(tags_unique)
skills = { p: i for i, p in enumerate(tags_unique) }
print("number of skills: %d" % num_skill)
np.save('../../data/ednet/map/cMap.npy',skills)
print(skills)

number of skills: 188
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 

In [8]:
# 建立练习-技能-邻接矩阵
adj_problem_skill=np.zeros((num_problem+1,num_skill))
for i,row in problem_detail.iterrows():
    problem_id=row['question_id']
    tags=row['tags']
    tag_list=tags.split(';')
    for tag in tag_list:
        tag=int(tag)    
        adj_problem_skill[problem_id][skills[tag]]=1
#保存e2c邻接矩阵
np.save(basePath+'adj/e2cAdj.npy',adj_problem_skill)

In [9]:
record_path=basePath+'EdNet/EdNet-KT1/KT1/'
files=os.listdir(record_path)
total_num_file=len(files)
print('总共',total_num_file,'个用户记录文件')

总共 784309 个用户记录文件


In [10]:
sample_stu=random.sample(files,5000)
print(sample_stu)
print('随机抽取其中',len(sample_stu),'个用户')
np.save(basePath+'raw/sample_stu.npy',sample_stu)

['u553078.csv', 'u162834.csv', 'u129845.csv', 'u155962.csv', 'u581376.csv', 'u541462.csv', 'u561531.csv', 'u35150.csv', 'u224434.csv', 'u700326.csv', 'u439419.csv', 'u80026.csv', 'u598601.csv', 'u21992.csv', 'u169009.csv', 'u602862.csv', 'u825993.csv', 'u582003.csv', 'u540288.csv', 'u810898.csv', 'u169330.csv', 'u463777.csv', 'u62292.csv', 'u36708.csv', 'u244765.csv', 'u745177.csv', 'u270430.csv', 'u371475.csv', 'u625759.csv', 'u355467.csv', 'u392575.csv', 'u166328.csv', 'u553486.csv', 'u270220.csv', 'u824662.csv', 'u482364.csv', 'u334684.csv', 'u162411.csv', 'u1951.csv', 'u574023.csv', 'u218351.csv', 'u320882.csv', 'u219667.csv', 'u440087.csv', 'u298833.csv', 'u456971.csv', 'u648790.csv', 'u459729.csv', 'u378834.csv', 'u275217.csv', 'u671372.csv', 'u362659.csv', 'u124420.csv', 'u265474.csv', 'u429819.csv', 'u164177.csv', 'u95464.csv', 'u58265.csv', 'u374766.csv', 'u508032.csv', 'u81073.csv', 'u208045.csv', 'u565041.csv', 'u114123.csv', 'u670157.csv', 'u524450.csv', 'u834154.csv', 'u24

In [11]:
kf = KFold(n_splits=5,shuffle=True)
allFoldStuFileNames=[]
for train_index,test_index in kf.split(sample_stu):
    allFoldStuFileNames.append((np.array(sample_stu)[train_index], np.array(sample_stu)[test_index]))

In [12]:
np.save(basePath+'raw/allFoldStuFileNames.npy',allFoldStuFileNames)

/home/hejunliang/.conda/envs/hjlTorch1/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [13]:
allFoldSeq=[]
for fold in allFoldStuFileNames:
    train_fold=[]
    test_fold=[]
    for train_stu in tqdm.tqdm(fold[0],'train stu'):
        stu_seq=pd.read_csv(basePath+'EdNet/EdNet-KT1/KT1/'+train_stu,usecols=['question_id','user_answer'])
        stu_seq['question_id']=stu_seq['question_id'].map(problems)
        stu_seq=pd.merge(stu_seq, problem_detail,on='question_id',how='left')
        stu_seq['correct']=(stu_seq['user_answer']==stu_seq['correct_answer']).astype(int)
        eid=stu_seq['question_id'].values
        correct=stu_seq['correct'].values
        train_fold.append((eid,correct))
    for test_stu in tqdm.tqdm(fold[1],'test stu'):
        stu_seq=pd.read_csv(basePath+'EdNet/EdNet-KT1/KT1/'+test_stu,usecols=['question_id','user_answer'])
        stu_seq['question_id']=stu_seq['question_id'].map(problems)
        stu_seq=pd.merge(stu_seq, problem_detail,on='question_id',how='left')
        stu_seq['correct']=(stu_seq['user_answer']==stu_seq['correct_answer']).astype(int)
        eid=stu_seq['question_id'].values
        correct=stu_seq['correct'].values
        test_fold.append((eid,correct))
    allFoldSeq.append((train_fold,test_fold))
np.save(basePath+'raw/allFoldSeq.npy',allFoldSeq)


test stu: 100%|██████████| 1000/1000 [00:06<00:00, 150.90it/s]


In [14]:
def splitToMaxStep(sequences,maxstep):
    e_data=[]
    a_data=[]
    for e_features,a in tqdm.tqdm(sequences, 'splitting into MaxStep: '):
        length=e_features.shape[0]
        slices = length//maxstep + (1 if length%maxstep > 0 else 0)
        for i in range(slices):
            e_temp = np.zeros(shape=[maxstep,1])
            a_temp = np.zeros(shape=[maxstep,1])
            if length>0:
                if length>=maxstep:
                    l=maxstep
                else:
                    l=length
                for j in range(l):
                    e_temp[j]=e_features[i*maxstep+j]
                    a_temp[j]=a[i*maxstep+j]
                length = length - maxstep
            e_data.append(e_temp)
            a_data.append(a_temp)
    
    return np.concatenate((np.array(e_data).astype(float),np.array(a_data).astype(float)),axis=2)

In [15]:
import _pickle as pickle
import numpy as np
allFoldSeq=np.load(basePath+'raw/allFoldSeq.npy',allow_pickle=True)
MAX_STEP = 128
i=1
for train_sequences,test_sequences in allFoldSeq:
    train_data=splitToMaxStep(train_sequences,MAX_STEP)
    test_data=splitToMaxStep(test_sequences,MAX_STEP)
    trainFile=open(basePath+'raw/train/train_data'+'_'+str(i)+'.txt','wb')
    testFile=open(basePath+'raw/test/test_data'+'_'+str(i)+'.txt','wb')
    pickle.dump(train_data,trainFile)
    pickle.dump(test_data,testFile)
    trainFile.close()
    testFile.close()
    i=i+1

splitting into MaxStep: 100%|██████████| 1000/1000 [00:00<00:00, 11911.30it/s]
